# Image Scaling with Machine Learning
## Image production or Streaming

### Inputs:
    1) Number of image pairs
    2) dictionary of {size_label_A: [pixels_high, pixels_wide], 
                      size_label_B: [pixels_high, pixels_wide]}
    3) run directory
    4) output directory
    5) existing sets parameters.json (None for first run)
****
[PIL image](https://pillow.readthedocs.io/en/5.1.x/reference/Image.html#the-image-class) <br>
****
[GitHub scalygraphic](https://github.com/dlanier/scalygraphic/) <br>
### Code adapted from FlyingMachineFractal (artistic history) 
[github.io/FlyingMachineFractal ](https://dlanier.github.io/FlyingMachineFractal/) <br>
[FlyingMachineFractal on GitHub](https://github.com/dlanier/FlyingMachineFractal) <br>
### Technique adapted from KnowEnG (BD2K) project
[Parameterized directory structure management](https://github.com/KnowEnG/KnowEnG_Pipelines_Library/blob/master/knpackage/toolbox.py) <br>
[yaml file example](https://github.com/KnowEnG/Samples_Clustering_Pipeline/blob/master/data/run_files/BENCHMARK_4_SC_cc_nmf_parallel_shared.yml) <br>
[YAML depreciation !!!](https://github.com/yaml/pyyaml/wiki/PyYAML-yaml.load(input)-Deprecation) <br>
****
#### Rewrite HSV coloring
[FMF graphic_utility.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/graphic_utility.py) <br>
[FMF numcolorpy.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/numcolorpy.py) <br>
****

In [3]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
import warnings
warnings.filterwarnings('ignore')

### Better Idea *:->* generate parameters dictionary only - re-run at different scales.


In [5]:
#    ../src/im_scale_products.py
import os
import sys
import time
import hashlib
import inspect
import numpy as np
import pandas as pd
import PIL
import yaml
import json

sys.path.insert(0, '../src/')

import zplain as zp
import eq_iter
import deg_0_ddeq
# import numcolorpy as ncp
import impute_color as ncp

def get_run_parameters(full_file_name, run_directory, results_directory):
    """ Read input arguments from yaml file and localize run direcories
    
    Args:
        full_file_name:     yaml file - parameters needed to run an option
        run_directory:      directory where code may run & write temp files
        images_directory:   output for image files

    Returns:
        run_parameters:     python dict eg. {   name_a: value_a, 
                                                name_b: value_b }
    """
    with open(full_file_name, 'r') as fh:
        run_parameters = yaml.safe_load(fh)

    run_parameters["run_file"] = full_file_name
    run_parameters["run_directory"] = run_directory
    run_parameters["results_directory"] = results_directory

    return run_parameters

"""     Constants (lookups):   ala import deg_0_ddeq

        EQUS_DICT is an enumerated dictionary of the functions in module deg_0_ddeq
        EQUS_DICT_NAMED_IDX is a dictionary - name: number index of EQUS_DICT 
"""
EQUS_DICT = {k: v for k, v in enumerate(inspect.getmembers(deg_0_ddeq, inspect.isfunction))}
EQUS_DICT_NAMED_IDX = {v[0]: k for k, v in EQUS_DICT.items()}

def get_rand_eq_p_set():
    """ get a random equation and parameter set from the deg_0_ddeq module
    (No Args:)
    Returns:
        tuple:      (function_name, function_handle, parameter_set)
    """
    n = np.random.randint(0,len(EQUS_DICT),1)
    fcn_name, fcn = EQUS_DICT[n[0]]
    p = fcn(0.0, None)

    return (fcn_name, fcn, p)

def get_eq_by_name(fcn_name):
    """ Usage: function_handle = get_eq_by_name(fcn_name)
    Args:
        name of a function in deg_0_ddeq
    """
    if fcn_name in EQUS_DICT_NAMED_IDX:
        return EQUS_DICT[EQUS_DICT_NAMED_IDX[fcn_name]][1]
    else:
        return None
    
def get_random_domain(bounds_dict=None):
    """ Usage: 
    domain_dict = get_random_domain(h, w, bounds_dict)
    
    Args:
        bounds_dict:    min - max limits for keys eg.
                            CP_magnitude_limits = {'min': 0, 'max': 7}
                            ZM_limits           = {'min': np.finfo(float).eps, 'max': 2}
                            theta_limits        = {'min': 0, 'max':2 * np.pi}
                        
    Returns:
        domain_dict:    with keys:
                            center_point
                            zoom
                            theta
    """
    domain_dict = {}
    if bounds_dict is None:
        CP_magnitude_limits =   {'min': 0, 'max': 2}
        ZM_limits =             {'min': np.finfo(float).eps, 'max': 1}
        theta_limits =          {'min': 0, 'max':2 * np.pi}
    else:
        CP_magnitude_limits =   bounds_dict['CP_magnitude_limits']
        ZM_limits =             bounds_dict['ZM_limits']
        theta_limits =          bounds_dict['theta_limits']

    r = np.random.uniform(low=0.0, high=2*np.pi) * 0.0+1.0j
    m = np.random.uniform(low=CP_magnitude_limits['min'], high=CP_magnitude_limits['max'])
    domain_dict['center_point'] = m*np.exp(r)
    domain_dict['zoom'] = np.random.uniform(low=ZM_limits['min'], high=ZM_limits['max'])
    domain_dict['theta'] = np.random.uniform(low=theta_limits['min'], high=theta_limits['max'])
    
    return domain_dict

def sha256sum(s):
    """ Usage:  hash_key = sha256sum(input_str)
    """
    h  = hashlib.sha256()
    h.update(bytes(s, 'ascii'))
    
    return h.hexdigest()

def hash_parameters(domain_dict, fcn_name, p):
    """ Usage: hash_key = hash_parameters(domain_dict, fcn_name, p)
    """
    N_DEC = 15
    f = zp.get_frame_from_dict(domain_dict)
    s = zp.complex_frame_dict_to_string(f, N_DEC) + '\n' + fcn_name
    if isinstance(p, list):
        p_str = ''
        for p_n in p:
            p_str += zp.complex_to_string(p_n, N_DEC)
    else:
        p_str = zp.complex_to_string(p, N_DEC)
    
    s += p_str
    
    return sha256sum(s)

def get_im(ET, Z, Z0):
    """ Usage: I = get_im(ET, Z, Z0)
    """
    Zd, Zr, ETn = ncp.etg_norm(Z0, Z, ET)

    A = np.zeros((domain_dict['n_rows'],domain_dict['n_cols'],3))
    A[:,:,0] += ETn     # Hue
    A[:,:,1] += Zr      # Saturation
    A[:,:,2] += Zd      # Value
    I = PIL.Image.fromarray(np.uint8(A * 255), 'HSV').convert('RGB')
    
    return I

def get_gray_im(ET, Z, Z0, et_gray=False):
    """ Usage: I = get_im(ET, Z, Z0)
    """
    Zd, Zr, ETn = ncp.etg_norm(Z0, Z, ET)
    
    if et_gray:
        I = PIL.Image.fromarray(np.uint8(ETn * 255), 'L')
    else:
        I = PIL.Image.fromarray(np.uint8(Zd * 255), 'L')
    
    return I

def now_name(prefi_str=None, suffi_str=None):
    """ get a human readable time stamp name """
    t0 = time.time()
    t_dec = t0 - np.floor(t0)
    ahora_nombre = time.strftime("%a_%d_%b_%Y_%H_%M_%S", time.localtime()) 
    if prefi_str is None: prefi_str = ''
    if suffi_str is None: suffi_str = ''
        
    return prefi_str + '_' + ahora_nombre + suffi_str



## Example yaml file - generate a json file of non-dimensional image parameters

In [6]:
# import os
# create run_files dir if DNE
run_file_directory = '../data/run_files/'
if os.path.isdir(run_file_directory) == False:
    os.makedirs(run_file_directory)

In [ ]:
# %%writefile ../data/run_files/generate_json_nd_parameter_set.yml

method: gen_nd_pars                 # Available methods: gen_im_set gen_nd_pars, stream_ims_frm_nd_pars
    
number_of_image_sets: 100
results_directory:   ../results
it_max:              64
escape_dist_scale:         10

## Produce very large set of unique figure run parameters
[StackOverflow append json](https://stackoverflow.com/questions/12994442/how-to-append-data-to-a-json-file) <br>
```python
# Append JSON object to output file JSON array
fname = "somefile.txt"
if os.path.isfile(fname):
    # File exists
    with open(fname, 'a+') as outfile:
        outfile.seek(-1, os.SEEK_END)
        outfile.truncate()
        outfile.write(',')
        json.dump(data_dict, outfile)
        outfile.write(']')
else: 
    # Create file
    with open(fname, 'w') as outfile:
        array = []
        array.append(data_dict)
        json.dump(array, outfile)
```
### Else:
[]() <br>
```python
import sqlite3
import pandas as pd

# Create the connection
cnx = sqlite3.connect(r'C:\mydatabases\bigdata.db')

# create the dataframe from a query
df = pd.read_sql_query("SELECT * FROM userdata", cnx)
```

```bash
TypeError: Object of type complex128 is not JSON serializable
```

In [8]:

def get_non_dimensional_image_parameters_set(run_parameters):
    """ generate a json file of unique parameter sets for generating images and multiple scales
    ndip_dict, run_parameters = get_non_dimensional_image_parameters_set(run_parameters)
    """
    ndim_dict = {}
    
    if not 'json_file_name' in run_parameters:
        run_parameters['json_file_name'] = now_name('nd_par_set', '.json')
        
    number_of_image_sets = run_parameters['number_of_image_sets']
    results_directory = run_parameters['results_directory']
    it_max = run_parameters['it_max']
    escape_dist_scale = run_parameters['escape_dist_scale']
    json_file_name = run_parameters['json_file_name']
    
    for runrunrun in range(number_of_image_sets):
        fcn_name, eq, p = get_rand_eq_p_set()
        domain_dict = get_random_domain()
        domain_dict['it_max'] = it_max
        domain_dict['max_d'] = escape_dist_scale / domain_dict['zoom']
        
        hash_idx = hash_parameters(domain_dict, fcn_name, p)
        ndim_dict[hash_idx] = {'domain_dict': domain_dict, 'fcn_name': fcn_name, 'p': p}

    return ndim_dict, run_parameters


run_file_name = '../data/run_files/generate_json_nd_parameter_set.yml'
run_directory = '../../test_dir'
results_directory = '../../ImagePars'
run_parameters = get_run_parameters(run_file_name, run_directory, results_directory)

run_parameters['number_of_image_sets'] = 1000
for k, v in run_parameters.items():
    print('%40s: %s'%(k, v))
print()
t_start = time.time()
ndim_dict, run_parameters = get_non_dimensional_image_parameters_set(run_parameters)
tt = time.time() - t_start
print('%6i parameter sets in %0.2f seconds'%(run_parameters['number_of_image_sets'], tt))

                                  method: gen_nd_pars
                    number_of_image_sets: 1000
                       results_directory: ../../ImagePars
                                  it_max: 64
                       escape_dist_scale: 10
                                run_file: ../data/run_files/generate_json_nd_parameter_set.yml
                           run_directory: ../../test_dir

  1000 parameter sets in 0.19 seconds


```python
"""
domain_dict
fcn_name
p
"""
{'domain_dict': {'center_point': (0.9974570444592821+1.553447306052334j),
  'zoom': 0.8564438540984382,
  'theta': 3.3651668445664584,
  'it_max': 64,
  'max_d': 11.676188639974312},
 'fcn_name': 'IslaLace',
 'p': [0.444476893762, (0.508164683992+0.420921535772j)]}
```

In [ ]:
#                                           Replace complex numbers for json limitation
"""    {'scalygraphic_type': 'complex128', 
        'shape': [1,1], 
        'value': [r, c], 
        'n_dec': 12      }             """
def domain_dict_to_json_dict(domain_dict):
    json_dict = {}
    # recursively convert complex numbers to strings
    return json_dict


def json_dict_to_domain_dict(json_dict):
    domain_dict = {}
    return domain_dict


In [9]:
keeze = list(ndim_dict.keys())
ndim_dict[keeze[0]]

{'domain_dict': {'center_point': (0.9974570444592821+1.553447306052334j),
  'zoom': 0.8564438540984382,
  'theta': 3.3651668445664584,
  'it_max': 64,
  'max_d': 11.676188639974312},
 'fcn_name': 'IslaLace',
 'p': [0.444476893762, (0.508164683992+0.420921535772j)]}

In [10]:
ndim_dict[keeze[-1]]

{'domain_dict': {'center_point': (0.345776779464405+0.5385154273441588j),
  'zoom': 0.8264847476145545,
  'theta': 5.98403439934556,
  'it_max': 64,
  'max_d': 12.099436836387541},
 'fcn_name': 'IslaLace',
 'p': [0.444476893762, (0.508164683992+0.420921535772j)]}

In [29]:
for k, v in run_parameters.items():
    print('%40s: %s'%(k, v))

                                  method: gen_nd_pars
                    number_of_image_sets: 1000000
                       results_directory: ../../ImagePars
                                  it_max: 64
                       escape_dist_scale: 10
                                run_file: ../data/run_files/generate_json_nd_parameter_set.yml
                           run_directory: ../../test_dir
                          json_file_name: nd_par_set_Fri_20_Sep_2019_13_47_59.json


In [ ]:
#                                                               json no do dakine complex numbers 

test_dir = '../../images_json_test'
if not os.path.isdir(test_dir):
    os.makedirs(test_dir)
json_file_name = os.path.join(test_dir, run_parameters['json_file_name'])
json_file_name
# with open(json_file_name, 'w') as fh:
#     json.dump(ndip_dict, fh, indent=4, sort_keys=True)
#                                                               json no do dakine complex numbers 

In [40]:
keeze = list(ndip_dict.keys())
ndip_dict[keeze[0]]

{'domain_dict': {'center_point': (0.21436349590304063+0.3338513644034249j),
  'zoom': 0.7833952328522382,
  'theta': 2.900318483886338,
  'it_max': 64,
  'max_d': 12.764948752102212},
 'fcn_name': 'decPwrAFx',
 'p': [1.7724538509055159, 1.13761386, -0.11556857]}

In [30]:
%whos

Variable                                   Type        Data/Info
----------------------------------------------------------------
EQUS_DICT                                  dict        n=9
EQUS_DICT_NAMED_IDX                        dict        n=9
PIL                                        module      <module 'PIL' from '/Libr<...>ackages/PIL/__init__.py'>
autopep8                                   module      <module 'autopep8' from '<...>te-packages/autopep8.py'>
deg_0_ddeq                                 module      <module 'deg_0_ddeq' from '../src/deg_0_ddeq.py'>
eq_iter                                    module      <module 'eq_iter' from '../src/eq_iter.py'>
get_eq_by_name                             function    <function get_eq_by_name at 0x11a07fd08>
get_gray_im                                function    <function get_gray_im at 0x11ef5c158>
get_im                                     function    <function get_im at 0x11a07ff28>
get_non_dimensional_image_parameters_set   functi

In [7]:
# define for multiple iterations of next cell
hashy_list = []

In [8]:
cell_start_time = time.time()

n_2_do = 10

test_temporary_dir = '../../test_temporary_dir'
if os.path.isdir(test_temporary_dir) == False:
    os.makedirs(test_temporary_dir)

DISPLAY_IN_NOTEBOOK = False

small_scale = [128, 128]
large_scale = [255, 255]
output_directory = '../../ImagesFriday'
if os.path.isdir(output_directory) == False:
    os.makedirs(output_directory)
    

for k_do in range(n_2_do):
    fcn_name, eq, p = get_rand_eq_p_set()
    print('\n%3i) %s'%(k_do+1, fcn_name))
    domain_dict = get_random_domain()

    domain_dict['it_max'] = 64
    domain_dict['max_d'] = 10 / domain_dict['zoom']

    hash_idx = hash_parameters(domain_dict, fcn_name, p)
    if hash_idx in hashy_list:
        print('\n\n\t\tImpossible! But! Skipping:\n%s\n'%(hash_idx))
    else:
        hashy_list.append(hash_idx)
        print(hash_idx + '\n')
        domain_dict['n_rows'] = small_scale[0]
        domain_dict['n_cols'] = small_scale[1]

        domain_dict['dir_path'] = test_temporary_dir

        list_tuple = [(eq, (p))]

        t0 = time.time()
        ET, Z, Z0 = eq_iter.get_primitives(list_tuple, domain_dict)
        I = get_im(ET, Z, Z0)
        file_name = os.path.join(output_directory, hash_idx + 'small.jpg')
        I.save(file_name)
        if DISPLAY_IN_NOTEBOOK:
            display(I)

        domain_dict['n_rows'] = large_scale[0]
        domain_dict['n_cols'] = large_scale[1]

        ET, Z, Z0 = eq_iter.get_primitives(list_tuple, domain_dict)
        I = get_im(ET, Z, Z0)
        file_name = os.path.join(output_directory, hash_idx + 'large.jpg')
        I.save(file_name)
        
        print('%0.3f\t 2 images time'%(time.time() - t0))
        if DISPLAY_IN_NOTEBOOK:
            display(I)

tt = time.time() - cell_start_time
print('\n%i pairs written in %0.2f seconds'%(n_2_do, tt))


  1) RoyalZ
3cf79c2b513229d8c311340a25f3976d3c72dceb4bc44c7d8f518a015bd26b85

19.536	 2 images time

  2) ItchicuPpwrF
a034c8d32c943cf5c760f63d89613f89f4ec74955ad37b416107079a7de28935

18.285	 2 images time

  3) ElGato
fcf622ce3dd73e7a731cfb2233678a6e3889991810e858298b6bc279881d2c5c

14.145	 2 images time

  4) Nautuliz
b29d551e0a4ca6af3269eec955367b735b2eab093bad01bf4f9ff69e3cbede8a

3.651	 2 images time

  5) ElGato
c3b5de1d5ba1bf88fd2f94a120e53bb56d2a22f822095647165b4d91da854ecb

14.754	 2 images time

  6) starfish_ish_II
e752eb438c61385c10fabed1a66e6589b3f3d0c1007c5a9edcdc473b0ff8f7c5

19.098	 2 images time

  7) Nautuliz
8f30f23076d983da225d53cffcb68807d10b65dc9760e4c7bdb920a134c3bf68

2.615	 2 images time

  8) starfish_ish_II
ed4937b7f41b04b37eb2bd26ef2777a2606ac2c9cd16c37eafd89c3bfcce614f

20.294	 2 images time

  9) decPwrAFx
1d7a49a7330fc871f117390b20f0511eb4290162bacf68a4b40aba18e38da1d1

14.918	 2 images time

 10) IslaLace
a078b48ea50a7a14fe11e05cf907563ff5d23d937098d6e

## file nameing ?

In [9]:
do_da = now_name(prefi_str=fcn_name, suffi_str='.jpg')
print(do_da)
d0_dat = hashy_list[0] + '.jpg'
print(d0_dat)

IslaLace_Fri_20_Sep_2019_07_56_43.jpg
3cf79c2b513229d8c311340a25f3976d3c72dceb4bc44c7d8f518a015bd26b85.jpg


## coloring logic
    * HSV model - choose three result components
        * Hue        (red to red) (0, 1) * 255          == visible spectrum colors
        * Saturation (gray to solid color) (0, 1) * 255 == color intensity
        * Value      (black to white) (0, 1) * 255      == brightness
    * RGB conversion from HSV
        * portable for multiple file formats
        * norm for train - validate - test
    * Escape-time algorithm produces
        * Escape Time (integer matrix)
        * Final Vector (Z - Z0)
            * distance (float matrix)
            * rotation (float matrix)
****
### Coloring component normalization:


In [10]:
def view_component_color_range(Z0, Z, ET):
    # Examine color ranges - for HSV - RGB conversion 
    Zd_t, Zr_t, ETn_t = ncp.etg_norm(Z0, Z, ET)
    print('\nZd_t', np.max(Zd_t), np.min(Zd_t))
    print('Zr_t', np.max(Zr_t), np.min(Zr_t))
    print('ETn_t', np.max(ETn_t), np.min(ETn_t))